# Установка данных

In [ ]:
import subprocess
!mkdir -p data
!wget https://hsehack.hb.ru-msk.vkcs.cloud/geo_dataframe.csv -P data
!wget https://hsehack.hb.ru-msk.vkcs.cloud/train_users.csv -P data
!mkdir -p data/requests
for i in range(30):
    print(f"Downloading part {i}...")
    subprocess.call(["wget", f"https://hsehack.hb.ru-msk.vkcs.cloud/requests/part_{i}.parquet", "-q", "-P", "data/requests"])

In [63]:
import pandas as pd

dir = 'data'
geo = pd.read_csv(f'{dir}/geo_dataframe.csv')
users = pd.read_csv(f'{dir}/train_users.csv')

In [97]:
name = 'part_1'
req = pd.read_parquet(f'{dir}/requests/{name}.parquet')# [:1000000]
req

,timestamp,geo_id,referer,user_id,user_agent
0,1712157404,5107,https://domain_2238/path_130676,9939177,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,1712113618,2759,https://www.domain_2434/path_175610,1263137,Mozilla/5.0 (Linux; arm_64; Android 10; HRY-LX...
2,1712162880,646,https://domain_2238/path_3204,4116321,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
3,1712121830,2247,https://domain_2238/path_3204,5208549,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,1712148547,3427,https://domain_905/path_7950,7328521,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...
...,...,...,...,...,...
10000447,1712136804,3948,https://www.domain_2395/path_175610,16364400,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
10000448,1711947059,3173,https://domain_1654/path_132496,7183420,Mozilla/5.0 (Linux; Android 9; ZTE Blade A5 20...
10000449,1712134136,4674,https://www.domain_294/path_31500,669141,Mozilla/5.0 (Linux; Android 9; Redmi 6A) Apple...
10000450,1712074115,4402,https://domain_2238/path_35686,446262,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...


# Сводим все таблицы по user_id и geo_id

In [98]:
merged = pd.merge(req, users, on='user_id', how='left')
merged = merged[pd.notna(merged.gender) | pd.notna(merged.age)]
merged = pd.merge(merged, geo, on='geo_id', how='left')
merged

,timestamp,geo_id,referer,user_id,user_agent,gender,age,region_id,country_id
0,1712157404,5107,https://domain_2238/path_130676,9939177,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0.0,50.0,-1,172
1,1712113618,2759,https://www.domain_2434/path_175610,1263137,Mozilla/5.0 (Linux; arm_64; Android 10; HRY-LX...,0.0,45.0,212,40
2,1712162880,646,https://domain_2238/path_3204,4116321,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1.0,71.0,65,40
3,1712121830,2247,https://domain_2238/path_3204,5208549,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0.0,74.0,194,40
4,1711960891,3357,https://www.domain_2434/path_131576,12596359,Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...,0.0,44.0,257,40
...,...,...,...,...,...,...,...,...,...
4660631,1712032475,708,https://domain_2238/path_97861,3666287,Mozilla/5.0 (Linux; arm_64; Android 7.0; P00L)...,1.0,68.0,74,40
4660632,1711955225,1330,https://domain_1942/path_118061,16996421,Mozilla/5.0 (Linux; U; Android 11; en-US; SM-A...,1.0,30.0,-1,145
4660633,1712134136,4674,https://www.domain_294/path_31500,669141,Mozilla/5.0 (Linux; Android 9; Redmi 6A) Apple...,0.0,57.0,243,199
4660634,1712074115,4402,https://domain_2238/path_35686,446262,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,0.0,61.0,215,40


# Работаем с фичами

In [40]:
!pip install user_agents

In [7]:
from tqdm.auto import tqdm
from user_agents import parse

def extract_os_and_browser(user_agent_string):
    user_agent = parse(user_agent_string)
    browser = user_agent.browser.family
    os = user_agent.os.family
    return browser, os

In [99]:
domains = []
paths = []
is_bot = []
is_email = []
is_mobile = []
is_pc = []
is_tablet = []

for i, row in tqdm(merged.iterrows()):
    r = row["referer"]
    if r:
        domains.append(int(r[r.index("domain") + 7: r.rfind("/")]))
        paths.append(int(r[r.index("path") + 5:]))
    else:
        domains.append(-1)
        paths.append(-1)

    is_bot.append(int(user_agent.is_bot))
    is_email.append(int(user_agent.is_email_client))
    is_mobile.append(int(user_agent.is_mobile))
    is_pc.append(int(user_agent.is_pc))
    is_tablet.append(int(user_agent.is_tablet))


0it [00:00, ?it/s]

In [109]:
merged['domain'] = domains
merged['path'] = paths
merged['is_bot'] = is_bot
merged['is_email'] = is_email
merged['is_mobile'] = is_mobile
merged['is_pc'] = is_pc
merged['is_tablet'] = is_tablet
merged.head(3)
merged[['timestamp', 'geo_id', 'referer', 'user_id', 'gender',
       'age', 'region_id', 'country_id', 'domain', 'path', 'is_bot',
       'is_email', 'is_mobile', 'is_pc', 'is_tablet']]

,timestamp,geo_id,referer,user_id,gender,age,region_id,country_id,domain,path,is_bot,is_email,is_mobile,is_pc,is_tablet
0,1712157404,5107,https://domain_2238/path_130676,9939177,0.0,50.0,-1,172,2238,130676,0,0,1,0,0
1,1712113618,2759,https://www.domain_2434/path_175610,1263137,0.0,45.0,212,40,2434,175610,0,0,1,0,0
2,1712162880,646,https://domain_2238/path_3204,4116321,1.0,71.0,65,40,2238,3204,0,0,1,0,0
3,1712121830,2247,https://domain_2238/path_3204,5208549,0.0,74.0,194,40,2238,3204,0,0,1,0,0
4,1711960891,3357,https://www.domain_2434/path_131576,12596359,0.0,44.0,257,40,2434,131576,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660631,1712032475,708,https://domain_2238/path_97861,3666287,1.0,68.0,74,40,2238,97861,0,0,1,0,0
4660632,1711955225,1330,https://domain_1942/path_118061,16996421,1.0,30.0,-1,145,1942,118061,0,0,1,0,0
4660633,1712134136,4674,https://www.domain_294/path_31500,669141,0.0,57.0,243,199,294,31500,0,0,1,0,0
4660634,1712074115,4402,https://domain_2238/path_35686,446262,0.0,61.0,215,40,2238,35686,0,0,1,0,0


# Избавляемся от ботов и пустых доменов / путей. Удаляем избыточные колонки

In [101]:
filtered = merged[(merged.is_bot == 0) & (merged.domain != -1) & (merged.path != -1)].drop(['referer', 'user_agent', 'geo_id', 'is_bot'], axis=1)
# filtered = merged[(merged.is_bot == 0) & (merged.domain != -1) & (merged.path != -1)].drop(['referer', 'geo_id', 'is_bot'], axis=1)
filtered

,timestamp,user_id,gender,age,region_id,country_id,domain,path,is_email,is_mobile,is_pc,is_tablet
0,1712157404,9939177,0.0,50.0,-1,172,2238,130676,0,1,0,0
1,1712113618,1263137,0.0,45.0,212,40,2434,175610,0,1,0,0
2,1712162880,4116321,1.0,71.0,65,40,2238,3204,0,1,0,0
3,1712121830,5208549,0.0,74.0,194,40,2238,3204,0,1,0,0
4,1711960891,12596359,0.0,44.0,257,40,2434,131576,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4660631,1712032475,3666287,1.0,68.0,74,40,2238,97861,0,1,0,0
4660632,1711955225,16996421,1.0,30.0,-1,145,1942,118061,0,1,0,0
4660633,1712134136,669141,0.0,57.0,243,199,294,31500,0,1,0,0
4660634,1712074115,446262,0.0,61.0,215,40,2238,35686,0,1,0,0


# Делаем новую фичу path_count

In [102]:
filtered['path_count'] = filtered.groupby(['user_id', 'domain'])['path'].transform('size')
filtered

,timestamp,user_id,gender,age,region_id,country_id,domain,path,is_email,is_mobile,is_pc,is_tablet,path_count
0,1712157404,9939177,0.0,50.0,-1,172,2238,130676,0,1,0,0,1
1,1712113618,1263137,0.0,45.0,212,40,2434,175610,0,1,0,0,2
2,1712162880,4116321,1.0,71.0,65,40,2238,3204,0,1,0,0,2
3,1712121830,5208549,0.0,74.0,194,40,2238,3204,0,1,0,0,1
4,1711960891,12596359,0.0,44.0,257,40,2434,131576,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660631,1712032475,3666287,1.0,68.0,74,40,2238,97861,0,1,0,0,2
4660632,1711955225,16996421,1.0,30.0,-1,145,1942,118061,0,1,0,0,3
4660633,1712134136,669141,0.0,57.0,243,199,294,31500,0,1,0,0,6
4660634,1712074115,446262,0.0,61.0,215,40,2238,35686,0,1,0,0,1


In [103]:
filtered.describe()

,timestamp,user_id,gender,age,region_id,country_id,domain,path,is_email,is_mobile,is_pc,is_tablet,path_count
count,4.638624e+06,4.638624e+06,4.638624e+06,4.638624e+06,4.638624e+06,4.638624e+06,4.638624e+06,4.638624e+06,4638624.0,4638624.0,4638624.0,4638624.0,4.638624e+06
mean,1.712054e+09,8.798642e+06,5.197009e-01,5.207442e+01,1.387705e+02,4.648728e+01,1.784507e+03,1.257225e+05,0.0,1.0,0.0,0.0,1.711112e+00
std,7.634040e+04,5.079699e+06,4.996118e-01,1.444002e+01,7.352825e+01,3.063434e+01,7.730426e+02,5.992503e+04,0.0,0.0,0.0,0.0,1.511111e+00
min,1.711919e+09,3.000000e+00,0.000000e+00,1.400000e+01,-1.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,0.0,1.0,0.0,0.0,1.000000e+00
25%,1.711980e+09,4.394524e+06,0.000000e+00,4.200000e+01,7.400000e+01,4.000000e+01,1.211000e+03,7.850000e+04,0.0,1.0,0.0,0.0,1.000000e+00
50%,1.712054e+09,8.810313e+06,1.000000e+00,5.300000e+01,1.750000e+02,4.000000e+01,2.042000e+03,1.525740e+05,0.0,1.0,0.0,0.0,1.000000e+00
75%,1.712130e+09,1.319468e+07,1.000000e+00,6.400000e+01,1.890000e+02,4.000000e+01,2.238000e+03,1.756100e+05,0.0,1.0,0.0,0.0,2.000000e+00
max,1.712178e+09,1.758886e+07,1.000000e+00,7.500000e+01,2.770000e+02,2.030000e+02,3.378000e+03,2.003810e+05,0.0,1.0,0.0,0.0,7.700000e+01


In [104]:
filtered.to_parquet(f"with_type2.parquet")

# Polars(((

In [16]:
import polars as pl


pl_filtered = pl.from_pandas(filtered)

In [19]:
pl_filtered = pl_filtered.filter(pl.col('domain').str != '')

In [37]:
pl_grouped = pl_filtered.group_by(['user_id', 'domain']).agg(pl.count())
pl_grouped.head()

user_id,domain,count
i32,str,u32
12787875,"""3191""",1
8871292,"""551""",2
11372673,"""662""",1
7906546,"""3191""",1
16922326,"""3149""",1


In [38]:
pl_final = pl_filtered.join(pl_grouped, on=['user_id', 'domain'])
pl_final.head()

timestamp,user_id,gender,age,region_id,country_id,domain,path,count
i64,i32,f64,f64,i64,i64,str,str,u32
1711945301,1013613,1.0,75.0,135,199,"""3207""","""175610""",1
1711941903,17082498,0.0,72.0,78,40,"""2042""","""144480""",1
1712088484,12787875,1.0,73.0,-1,79,"""3191""","""9105""",1
1711918808,8871292,0.0,18.0,192,40,"""551""","""175610""",2
1712117806,11372673,1.0,41.0,184,40,"""662""","""120157""",1


In [39]:
final = pl_final.to_pandas()
final.head()

,timestamp,user_id,gender,age,region_id,country_id,domain,path,count
0,1711945301,1013613,1.0,75.0,135,199,3207,175610,1
1,1711941903,17082498,0.0,72.0,78,40,2042,144480,1
2,1712088484,12787875,1.0,73.0,-1,79,3191,9105,1
3,1711918808,8871292,0.0,18.0,192,40,551,175610,2
4,1712117806,11372673,1.0,41.0,184,40,662,120157,1


In [34]:
final['count'].value_counts()

count
1    44842
2     3534
3      534
4       84
5       20
6        6
Name: count, dtype: int64

In [ ]:
import polars as pl


pl_filtered = pl.from_pandas(filtered)
pl_filtered = pl_filtered.filter(pl.col('domain').str != '')
pl_grouped = pl_filtered.group_by(['user_id', 'domain']).agg(pl.count())
pl_final = pl_filtered.join(pl_grouped, on=['user_id', 'domain'])
final = pl_final.to_pandas()